# First part (10 marks)

In [1]:
!conda install -c anaconda lxml -y
!conda install -c anaconda html5lib -y
!conda install -c anaconda BeautifulSoup4 -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda
    ca-certificates-2019.8.28  |                0         132 KB  anaconda
    lxml-4.4.1                 |   py36hefd8a0e_0         1.6 MB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.9 MB

The following packages will be UPDATED:

    ca-certificates: 2019.5.15-1          --> 2019.8.28-0          anaconda
    certifi:         2019.6.16-py36_1     --> 2019.9.11-py36_0     anaconda
    lxml:            4.3.1-py36hefd8a0e_0 --> 4.4.1-py36hefd8a0e_0 anaconda
    openssl:        

In [1]:
import pandas as pd
import numpy as np
import html5lib
import requests
from pandas.io.json import json_normalize

### Hello Capstone Project Course!

In [2]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_canada = table[0]
df_canada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [198]:
df_canada.shape

(210, 3)

In [194]:
# оставляем строки с значением в столбцу ['Borough'] не равном 'Not assigned'
df_canada = df_canada.loc[df_canada['Borough'] != 'Not assigned']
df_canada = df_canada.loc[df_canada['Neighbourhood'] != 'Not assigned']

In [204]:
# группировка по столбцу ['Postcode'] с возвращением пересечения по строкам в каждом столбце

df_canada_new = df_canada.groupby('Postcode').agg(lambda x: list(set(x)))
df_canada_new = df_canada_new.reset_index()
# преобразование списков в столбцах фрейма в строки
for i in range(len(df_canada_new)):
    df_canada_new['Neighbourhood'][i] = ', '.join(df_canada_new['Neighbourhood'][i])
    df_canada_new['Borough'][i] = ', '.join(df_canada_new['Borough'][i])
df_canada_new.astype('object').dtypes

# альтернативный вариант
#df_canada_new = df_canada.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))

In [205]:
df_canada_new.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [206]:
df_canada_new.shape
# I don't know why 102 rows 

(102, 3)

#  I don't know why 102 rows, it's not the code.

# Second part (2 marks)

In [7]:
API_KEY = 'AIzrSyB3mv1cYRL9tec1VDNJgBXDX1y3MTV8w2A'

In [36]:
coor = []
for postcode in df_canada_new['Postcode'].values:
    try:
        results = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?address={postcode},CA&key={API_KEY}').json()
        coor.append(results['results'][0]['geometry']['location'])
    except:
        coor.append({'lat': 'NaN', 'lng': 'NaN'})

In [48]:
df_coor = pd.DataFrame(coor)
df_coor.shape

(102, 2)

In [49]:
#объединение двух фреймов 
df_canada_new = pd.concat([df_canada_new, df_coor], axis = 1)

In [50]:
df_canada_new.head()

,Postcode,Borough,Neighbourhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


# Third part (2 marks)

In [55]:
df_toronto = df_canada_new[df_canada_new['Borough'].str.contains("Toronto")]
df_toronto.shape

(38, 5)

In [63]:
#df.loc[df['y'] == 'N/A','y'] = np.nan
df_toronto.loc[df_toronto['lat'] == 'NaN'] = np.nan
df_toronto.loc[df_toronto['lng'] == 'NaN'] = np.nan
df_toronto = df_toronto.dropna(subset = ['lat'])
df_toronto.shape

(36, 5)

In [73]:
df_toronto.head()

,Postcode,Borough,Neighbourhood,lat,lng
37,M4E,East Toronto,The Beaches,43.6764,-79.293
41,M4K,East Toronto,"Riverdale, The Danforth West",43.6796,-79.3522
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.669,-79.3156
43,M4M,East Toronto,Studio District,33.7758,-118.171
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888


In [60]:
# проверка на наличие NaN
missing_data = df_toronto.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Postcode
False    36
Name: Postcode, dtype: int64

Borough
False    36
Name: Borough, dtype: int64

Neighbourhood
False    36
Name: Neighbourhood, dtype: int64

lat
False    36
Name: lat, dtype: int64

lng
False    36
Name: lng, dtype: int64



In [68]:
#address = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?address=Toronto,CA&key={API_KEY}').json()

In [70]:
address['results'][0]['geometry']['location']
latitude =  address['results'][0]['geometry']['location']['lat']
longitude = address['results'][0]['geometry']['location']['lng']
print(latitude, longitude)

43.653226 -79.3831843


In [69]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [77]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_toronto['lat'], df_toronto['lng'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [78]:
CLIENT_ID = 'UYMSGED40GUNNEYFX9BRJCWAADUDFBSM2NG0OR0EWQ2DHMLW' 
CLIENT_SECRET = '5HA1JHCCPN0LBLM4YYOSXM10L9PMRSYE53PCKLRKSYZE3ER' 
VERSION = '20180605'

In [79]:
# функция для получения топ-100 заведений в каждом районе Торонто
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API запрос на ресурс Foursquare
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # добавление в список только необходимой информации из json
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    # создание фрейма из списка
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [80]:
LIMIT = 100
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['lat'],
                                   longitudes=df_toronto['lng']
                                  )

The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, South Hill, Forest Hill SE, Rathnelly, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
King, Adelaide, Richmond
Harbourfront East, Union Station, Toronto Islands
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Chinatown, Grange Park, Kensington Market
South Niagara, Railway Lands, CN Tower, Island airport, Harbourfront West, King and Spadina, Bathurst Quay
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dufferin, Dovercourt Village
Parkdale Village, Brockton, Exhibition Place
The Junction South, High Park
Parkdale, 

In [122]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors


In [137]:
# каждая ктегория из колонки с категориями мест трансформируется в отдельную колонку с 1 в районах где есть эта ктегория и 0 где нет
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# определяем порядок колоннок в фрейме, каго-то хера колонка ['Neighborhood'] добовляется не в конец а по алфавиту на 154 индекс((((
fixed_columns = [toronto_onehot.columns[154]] + list(toronto_onehot.columns[:154]) + list(toronto_onehot.columns[156:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
#группировка по районам
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.017544,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.066667
2,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011765,0.0,...,0.0,0.0,0.011765,0.0,0.0,0.00,0.011765,0.0,0.0,0.011765
4,"Chinatown, Grange Park, Kensington Market",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.060000,0.0,0.0,0.04,0.010000,0.0,0.0,0.000000


In [139]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3            Beer Bar  0.04
4      Farmers Market  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.07
1     Garden Center  0.07
2        Comic Shop  0.07
3              Park  0.07
4  Recording Studio  0.07


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1                 Pub  0.05
2  Italian Restaurant  0.05
3         Pizza Place  0.05
4                Park  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4        Burger Joint  0.04


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0                           Café  0.07
1  Vegetarian / Vegan Restaurant  0.06
2             Chinese Re

In [140]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Steakhouse,Bakery,Beer Bar,Seafood Restaurant,Café,Cheese Shop,Jazz Club
1,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market,Comic Shop
2,"Cabbagetown, St. James Town",Coffee Shop,Pub,Italian Restaurant,Park,Pizza Place,Bakery,Café,Restaurant,Market,Breakfast Spot
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Sandwich Place,Ice Cream Shop,Chinese Restaurant,Bubble Tea Shop,Spa,Bar
4,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Mexican Restaurant,Coffee Shop,Park


In [148]:
neighborhoods_venues_sorted['1st Most Common Venue'].value_counts()

Coffee Shop             13
Fast Food Restaurant     3
Park                     3
Café                     3
Sandwich Place           2
Health Food Store        1
Garden                   1
Breakfast Spot           1
Yoga Studio              1
Playground               1
Photography Studio       1
Liquor Store             1
Food & Drink Shop        1
Grocery Store            1
Pharmacy                 1
Greek Restaurant         1
Airport Terminal         1
Name: 1st Most Common Venue, dtype: int64

In [150]:
from sklearn.cluster import KMeans

In [185]:
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(init="k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
del neighborhoods_venues_sorted['Cluster Labels']

In [186]:
# add clustering labele

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6764,-79.293,0,Health Food Store,Park,Other Great Outdoors,Trail,Pub,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,M4K,East Toronto,"Riverdale, The Danforth West",43.6796,-79.3522,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Café
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.669,-79.3156,0,Sandwich Place,Pizza Place,Pet Store,Pub,Liquor Store,Light Rail Station,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
43,M4M,East Toronto,Studio District,33.7758,-118.171,0,Liquor Store,Pet Store,Dive Bar,Boutique,Breakfast Spot,Pub,Chinese Restaurant,Pizza Place,Convenience Store,Music Store
44,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,0,Photography Studio,Park,Swim School,Bus Line,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [187]:
toronto_merged['Cluster Labels'].value_counts()

0    28
2     4
1     4
Name: Cluster Labels, dtype: int64

In [177]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['lng'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [190]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Central Toronto,1,Fast Food Restaurant,Home Service,Burger Joint,Ice Cream Shop,Jewelry Store,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space
61,Downtown Toronto,1,Fast Food Restaurant,Home Service,Burger Joint,Ice Cream Shop,Jewelry Store,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space
63,Central Toronto,1,Garden,Home Service,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
66,Downtown Toronto,1,Fast Food Restaurant,Home Service,Burger Joint,Ice Cream Shop,Jewelry Store,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Event Space


# 1 observation

most common venue in Toronto are Cafe or Coffee Shop

# 2 observation

the pest k for clustering is 3